### Import libraries and make an api call to data.gov.au to fetch the volunteers data
The returned reults are in json format, this is converted to a pandas dataframe for ease of use

In [1]:
import requests
import urllib.request, json
from pandas.io.json import json_normalize
import pandas as pd
import geocoder

url = 'https://data.gov.au/data/api/3/action/datastore_search?resource_id=c9be70dc-a029-4bdc-b2f9-ec5f5f780d0c'  

r_json = requests.get(url).json()
df = json_normalize(r_json)
df.head()

,help,result._links.next,result._links.start,result.fields,result.include_total,result.records,result.records_format,result.resource_id,result.total,success
0,https://data.gov.au/data/api/3/action/help_sho...,/api/3/action/datastore_search?offset=100&reso...,/api/3/action/datastore_search?resource_id=c9b...,"[{'type': 'int', 'id': '_id'}, {'type': 'text'...",True,"[{'_id': 1, 'Name': 'Albury Wodonga Volunteer ...",objects,c9be70dc-a029-4bdc-b2f9-ec5f5f780d0c,93,True


### Reshape data
The data in the dataframe does not have the desired shape, hence it is reshaped to fit the required format, only have the essential data in the dataframe

In [2]:
df = pd.concat({i:pd.DataFrame(x) for i, x in df.pop('result.records').items()}).reset_index(level=1, drop=True)
df.head()

,Address,Business Category,LGA,Name,Postcode,Region,State,Suburb,_id
0,PO Box 730,Volunteering,Wodonga (RC),Albury Wodonga Volunteer Resource Centre,3690,Hume Region,VIC,Wodonga,1
0,17 Young Crescent,Volunteering,Alpine (S),Alpine U3A Inc,3698,Hume Region,VIC,Tawonga South,2
0,PO Box 2058,Volunteering,None,Ballarat U3A Inc.,3354,None,VIC,Bakery Hill,3
0,16 Sturt Street,Volunteering,Ballarat,Ballarat Volunteer Resource Centre,3353,Grampians,VIC,Ballarat,4
0,PO Box 1320,Volunteering,Baw Baw (S),Baw Baw U3A Inc.,3820,Gippsland Region,VIC,Warragul,5


### Basic analysis to look for any anomalies in the data

In [3]:
df

,Address,Business Category,LGA,Name,Postcode,Region,State,Suburb,_id
0,PO Box 730,Volunteering,Wodonga (RC),Albury Wodonga Volunteer Resource Centre,3690,Hume Region,VIC,Wodonga,1
0,17 Young Crescent,Volunteering,Alpine (S),Alpine U3A Inc,3698,Hume Region,VIC,Tawonga South,2
0,PO Box 2058,Volunteering,None,Ballarat U3A Inc.,3354,None,VIC,Bakery Hill,3
0,16 Sturt Street,Volunteering,Ballarat,Ballarat Volunteer Resource Centre,3353,Grampians,VIC,Ballarat,4
0,PO Box 1320,Volunteering,Baw Baw (S),Baw Baw U3A Inc.,3820,Gippsland Region,VIC,Warragul,5
0,PO Box 374,Volunteering,Greater Bendigo,Bendigo Volunteer Resource Centre,3552,Loddon-Mallee,VIC,Bendigo Central,6
0,405 Camberwell Road,Volunteering,Boroondara,Boroondara Volunteer Resource Centre,3124,Eastern Metro,VIC,Camberwell,7
0,PO Box 458,Volunteering,Alpine (S),Bright District U3A Inc.,3741,Hume Region,VIC,Bright,8
0,"Fiddler's Green Retirement Village, 57 Glouces...",Volunteering,Casey,Casey U3A Inc.,3806,Southern Metro,VIC,Berwick,9
0,"Shop 111, Epping Plaza, Cnr Cooper & High Streets",Volunteering,Whittlesea,Community Information Whittlesea,3076,North and West Metropolitan Region,VIC,Epping,10


In [4]:
df[df.Region.isna()]

,Address,Business Category,LGA,Name,Postcode,Region,State,Suburb,_id
0,PO Box 2058,Volunteering,None,Ballarat U3A Inc.,3354,None,VIC,Bakery Hill,3
0,Mernda Community House,Volunteering,None,Whittlesea U3A Inc,3754,None,VIC,Mernda,82


In [5]:
df.dtypes

Address              object
Business Category    object
LGA                  object
Name                 object
Postcode              int64
Region               object
State                object
Suburb               object
_id                   int64
dtype: object

### Extract the suburbs of melbourne data to retain only those suburbs of victoria that belong to the Greater Melbourne Region (GMR)

In [6]:
gmr_sub = pd.read_csv("../Data/Suburbs_of_mel.csv")

In [7]:
gmr_sub.head()

,Locality Name,Post Code,Municipality Name,Municipality
0,Abbotsford,3067,Yarra City Council,Yarra
1,Alphington,3078,Yarra City Council,Yarra
2,Burnley,3121,Yarra City Council,Yarra
3,Carlton North,3054,Yarra City Council,Yarra
4,Clifton Hill,3068,Yarra City Council,Yarra


### Merge the 2 dataframes to only retain records in volunteering that belong to GMR

In [8]:
volunteer_mel = pd.merge(df, gmr_sub, how = 'inner', left_on ="Suburb", right_on = "Locality Name")
volunteer_mel.head()

,Address,Business Category,LGA,Name,Postcode,Region,State,Suburb,_id,Locality Name,Post Code,Municipality Name,Municipality
0,405 Camberwell Road,Volunteering,Boroondara,Boroondara Volunteer Resource Centre,3124,Eastern Metro,VIC,Camberwell,7,Camberwell,3124,Boroondara City Council,Boroondara
1,"Fiddler's Green Retirement Village, 57 Glouces...",Volunteering,Casey,Casey U3A Inc.,3806,Southern Metro,VIC,Berwick,9,Berwick,3806,Casey City Council,Casey
2,"Shop 111, Epping Plaza, Cnr Cooper & High Streets",Volunteering,Whittlesea,Community Information Whittlesea,3076,North and West Metropolitan Region,VIC,Epping,10,Epping,3076,Whittlesea City Council,Whittlesea
3,"Community Information Whittlesea, Shop 111, Ep...",Volunteering,Whittlesea,Whittlesea Volunteer Resource Service,3076,North and West Metropolitan Region,VIC,Epping,83,Epping,3076,Whittlesea City Council,Whittlesea
4,214 Mount Dandenong Road,Volunteering,Maroondah,Croydon U3A Inc.,3136,Eastern Metro,VIC,Croydon,11,Croydon,3136,Maroondah City Council,Maroondah


### Drop unwanted columns and rename columns to better suit the standard format

In [9]:
volunteer_mel.drop(columns=["LGA","Locality Name", "Post Code", "Municipality", "Region", "State", "_id"], inplace = True)
volunteer_mel.rename(columns = {"Municipality Name": "Municipality"}, inplace = True)
volunteer_mel.reset_index(drop = True, inplace=True)
volunteer_mel

,Address,Business Category,Name,Postcode,Suburb,Municipality
0,405 Camberwell Road,Volunteering,Boroondara Volunteer Resource Centre,3124,Camberwell,Boroondara City Council
1,"Fiddler's Green Retirement Village, 57 Glouces...",Volunteering,Casey U3A Inc.,3806,Berwick,Casey City Council
2,"Shop 111, Epping Plaza, Cnr Cooper & High Streets",Volunteering,Community Information Whittlesea,3076,Epping,Whittlesea City Council
3,"Community Information Whittlesea, Shop 111, Ep...",Volunteering,Whittlesea Volunteer Resource Service,3076,Epping,Whittlesea City Council
4,214 Mount Dandenong Road,Volunteering,Croydon U3A Inc.,3136,Croydon,Maroondah City Council
5,32 Greenwood Avenue,Volunteering,Eastern Volunteer Resource Centre,3134,Ringwood,Maroondah City Council
6,"Emerald Community House, Main Road",Volunteering,Emerald U3A Inc.,3782,Emerald,Cardinia Shire Council
7,"Emerald Community House, Main Road",Volunteering,Emerald U3A Inc.,3782,Emerald,Yarra Ranges Shire Council
8,PO Box 287,Volunteering,Endowment Australasia,3084,Rosanna,Banyule City Council
9,1151 Glen Huntly Road,Volunteering,Glen Eira U3A Inc.,3163,Glen Huntly,Glen Eira City Council


### Adding co-ordinates data

In [17]:
f = open('../Data/key.txt',"r")

In [18]:
key = f.read()

In [11]:
volunteer_mel["Address"] = [volunteer_mel["Address"][i]+", "+volunteer_mel["Suburb"][i]+", Victoria "+ str(volunteer_mel["Postcode"][i]) for i in range(len(volunteer_mel))]
lat = []
lon = []
for i in range(len(volunteer_mel)):
    g = geocoder.bing(volunteer_mel["Address"][i], key=key)
    lat.append(g.json['lat'])
    lon.append(g.json['lng'])

volunteer_mel["Latitude"] = lat
volunteer_mel["Longitude"] = lon

In [12]:
volunteer_mel

,Address,Business Category,Name,Postcode,Suburb,Municipality,Latitude,Longitude
0,"405 Camberwell Road, Camberwell, Victoria 3124",Volunteering,Boroondara Volunteer Resource Centre,3124,Camberwell,Boroondara City Council,-37.835570,145.062190
1,"Fiddler's Green Retirement Village, 57 Glouces...",Volunteering,Casey U3A Inc.,3806,Berwick,Casey City Council,-38.037830,145.345750
2,"Shop 111, Epping Plaza, Cnr Cooper & High Stre...",Volunteering,Community Information Whittlesea,3076,Epping,Whittlesea City Council,-37.651570,145.016490
3,"Community Information Whittlesea, Shop 111, Ep...",Volunteering,Whittlesea Volunteer Resource Service,3076,Epping,Whittlesea City Council,-37.627780,145.027949
4,"214 Mount Dandenong Road, Croydon, Victoria 3136",Volunteering,Croydon U3A Inc.,3136,Croydon,Maroondah City Council,-37.799000,145.281600
5,"32 Greenwood Avenue, Ringwood, Victoria 3134",Volunteering,Eastern Volunteer Resource Centre,3134,Ringwood,Maroondah City Council,-37.819720,145.230240
6,"Emerald Community House, Main Road, Emerald, V...",Volunteering,Emerald U3A Inc.,3782,Emerald,Cardinia Shire Council,-37.926380,145.443890
7,"Emerald Community House, Main Road, Emerald, V...",Volunteering,Emerald U3A Inc.,3782,Emerald,Yarra Ranges Shire Council,-37.926380,145.443890
8,"PO Box 287 , Rosanna, Victoria 3084",Volunteering,Endowment Australasia,3084,Rosanna,Banyule City Council,-37.742729,145.065277
9,"1151 Glen Huntly Road, Glen Huntly, Victoria 3163",Volunteering,Glen Eira U3A Inc.,3163,Glen Huntly,Glen Eira City Council,-37.888870,145.041820


### Add contact details

In [17]:
f = open('../Data/Subkey.txt',"r")

In [18]:
Skey = f.read()

In [36]:
import http.client, urllib.parse
import json

subscriptionKey = Skey

host = 'api.cognitive.microsoft.com'
path = '/bing/v7.0/search'

contact = []

def get_local(params):
    headers = {'Ocp-Apim-Subscription-Key': subscriptionKey}
    conn = http.client.HTTPSConnection (host)
    conn.request ("GET", path + params, None, headers)
    response = conn.getresponse ()
    return response.read ()

for i in range(len(volunteer_mel)):
    query = volunteer_mel["Name"][i]
    params = '?q=' + urllib.parse.quote (query) + '&mkt=en-us'
    result = get_local(params)
    contact.append(json.loads(result)["webPages"]["value"][0]["displayUrl"])

volunteer_mel["Contact"] = contact

In [37]:
volunteer_mel

,Address,Business Category,Name,Postcode,Suburb,Municipality,Latitude,Longitude,Categories,Contact
0,"405 Camberwell Road, Camberwell, Victoria 3124",Volunteering,Boroondara Volunteer Resource Centre,3124,Camberwell,Boroondara City Council,-37.835570,145.062190,"[Boroondara City Council, Camberwell, Voluntee...",https://www.boroondara.vic.gov.au/.../about-bo...
1,"Fiddler's Green Retirement Village, 57 Glouces...",Volunteering,Casey U3A Inc.,3806,Berwick,Casey City Council,-38.037830,145.345750,"[Casey City Council, Berwick, Volunteering]",https://casey.u3anet.org.au/courses-and-classe...
2,"Shop 111, Epping Plaza, Cnr Cooper & High Stre...",Volunteering,Community Information Whittlesea,3076,Epping,Whittlesea City Council,-37.651570,145.016490,"[Whittlesea City Council, Epping, Volunteering]",https://www.whittleseacommunityconnections.org.au
3,"Community Information Whittlesea, Shop 111, Ep...",Volunteering,Whittlesea Volunteer Resource Service,3076,Epping,Whittlesea City Council,-37.627780,145.027949,"[Whittlesea City Council, Epping, Volunteering]",https://www.whittlesea.vic.gov.au/community-su...
4,"214 Mount Dandenong Road, Croydon, Victoria 3136",Volunteering,Croydon U3A Inc.,3136,Croydon,Maroondah City Council,-37.799000,145.281600,"[Maroondah City Council, Croydon, Volunteering]",https://u3acroydon.org.au
5,"32 Greenwood Avenue, Ringwood, Victoria 3134",Volunteering,Eastern Volunteer Resource Centre,3134,Ringwood,Maroondah City Council,-37.819720,145.230240,"[Maroondah City Council, Ringwood, Volunteering]",https://easternvolunteers.org.au
6,"Emerald Community House, Main Road, Emerald, V...",Volunteering,Emerald U3A Inc.,3782,Emerald,Cardinia Shire Council,-37.926380,145.443890,"[Cardinia Shire Council, Emerald, Volunteering]",https://u3aemerald.org.au
7,"Emerald Community House, Main Road, Emerald, V...",Volunteering,Emerald U3A Inc.,3782,Emerald,Yarra Ranges Shire Council,-37.926380,145.443890,"[Yarra Ranges Shire Council, Emerald, Voluntee...",https://u3aemerald.org.au
8,"PO Box 287 , Rosanna, Victoria 3084",Volunteering,Endowment Australasia,3084,Rosanna,Banyule City Council,-37.742729,145.065277,"[Banyule City Council, Rosanna, Volunteering]",https://myclaimsolved.com/.../national-mutual-...
9,"1151 Glen Huntly Road, Glen Huntly, Victoria 3163",Volunteering,Glen Eira U3A Inc.,3163,Glen Huntly,Glen Eira City Council,-37.888870,145.041820,"[Glen Eira City Council, Glen Huntly, Voluntee...",https://gleneirau3a.org.au


### Create a categories column to follow [Municipality, Suburb, Volunteering] format

In [22]:
volunteer_mel["Categories"] = [[volunteer_mel["Municipality"][i], volunteer_mel["Suburb"][i],"Volunteering"] for i in range(len(volunteer_mel))]
volunteer_mel

,Address,Business Category,Name,Postcode,Suburb,Municipality,Latitude,Longitude,Categories
0,"405 Camberwell Road, Camberwell, Victoria 3124",Volunteering,Boroondara Volunteer Resource Centre,3124,Camberwell,Boroondara City Council,-37.835570,145.062190,"[Boroondara City Council, Camberwell, Voluntee..."
1,"Fiddler's Green Retirement Village, 57 Glouces...",Volunteering,Casey U3A Inc.,3806,Berwick,Casey City Council,-38.037830,145.345750,"[Casey City Council, Berwick, Volunteering]"
2,"Shop 111, Epping Plaza, Cnr Cooper & High Stre...",Volunteering,Community Information Whittlesea,3076,Epping,Whittlesea City Council,-37.651570,145.016490,"[Whittlesea City Council, Epping, Volunteering]"
3,"Community Information Whittlesea, Shop 111, Ep...",Volunteering,Whittlesea Volunteer Resource Service,3076,Epping,Whittlesea City Council,-37.627780,145.027949,"[Whittlesea City Council, Epping, Volunteering]"
4,"214 Mount Dandenong Road, Croydon, Victoria 3136",Volunteering,Croydon U3A Inc.,3136,Croydon,Maroondah City Council,-37.799000,145.281600,"[Maroondah City Council, Croydon, Volunteering]"
5,"32 Greenwood Avenue, Ringwood, Victoria 3134",Volunteering,Eastern Volunteer Resource Centre,3134,Ringwood,Maroondah City Council,-37.819720,145.230240,"[Maroondah City Council, Ringwood, Volunteering]"
6,"Emerald Community House, Main Road, Emerald, V...",Volunteering,Emerald U3A Inc.,3782,Emerald,Cardinia Shire Council,-37.926380,145.443890,"[Cardinia Shire Council, Emerald, Volunteering]"
7,"Emerald Community House, Main Road, Emerald, V...",Volunteering,Emerald U3A Inc.,3782,Emerald,Yarra Ranges Shire Council,-37.926380,145.443890,"[Yarra Ranges Shire Council, Emerald, Voluntee..."
8,"PO Box 287 , Rosanna, Victoria 3084",Volunteering,Endowment Australasia,3084,Rosanna,Banyule City Council,-37.742729,145.065277,"[Banyule City Council, Rosanna, Volunteering]"
9,"1151 Glen Huntly Road, Glen Huntly, Victoria 3163",Volunteering,Glen Eira U3A Inc.,3163,Glen Huntly,Glen Eira City Council,-37.888870,145.041820,"[Glen Eira City Council, Glen Huntly, Voluntee..."


### Drop duplicates and write to file

In [38]:
volunteer_mel.drop_duplicates(subset=["Name"], inplace = True)
volunteer_mel.drop_duplicates(subset=["Address"], inplace = True)
volunteer_mel

,Address,Business Category,Name,Postcode,Suburb,Municipality,Latitude,Longitude,Categories,Contact
0,"405 Camberwell Road, Camberwell, Victoria 3124",Volunteering,Boroondara Volunteer Resource Centre,3124,Camberwell,Boroondara City Council,-37.835570,145.062190,"[Boroondara City Council, Camberwell, Voluntee...",https://www.boroondara.vic.gov.au/.../about-bo...
1,"Fiddler's Green Retirement Village, 57 Glouces...",Volunteering,Casey U3A Inc.,3806,Berwick,Casey City Council,-38.037830,145.345750,"[Casey City Council, Berwick, Volunteering]",https://casey.u3anet.org.au/courses-and-classe...
2,"Shop 111, Epping Plaza, Cnr Cooper & High Stre...",Volunteering,Community Information Whittlesea,3076,Epping,Whittlesea City Council,-37.651570,145.016490,"[Whittlesea City Council, Epping, Volunteering]",https://www.whittleseacommunityconnections.org.au
3,"Community Information Whittlesea, Shop 111, Ep...",Volunteering,Whittlesea Volunteer Resource Service,3076,Epping,Whittlesea City Council,-37.627780,145.027949,"[Whittlesea City Council, Epping, Volunteering]",https://www.whittlesea.vic.gov.au/community-su...
4,"214 Mount Dandenong Road, Croydon, Victoria 3136",Volunteering,Croydon U3A Inc.,3136,Croydon,Maroondah City Council,-37.799000,145.281600,"[Maroondah City Council, Croydon, Volunteering]",https://u3acroydon.org.au
5,"32 Greenwood Avenue, Ringwood, Victoria 3134",Volunteering,Eastern Volunteer Resource Centre,3134,Ringwood,Maroondah City Council,-37.819720,145.230240,"[Maroondah City Council, Ringwood, Volunteering]",https://easternvolunteers.org.au
6,"Emerald Community House, Main Road, Emerald, V...",Volunteering,Emerald U3A Inc.,3782,Emerald,Cardinia Shire Council,-37.926380,145.443890,"[Cardinia Shire Council, Emerald, Volunteering]",https://u3aemerald.org.au
8,"PO Box 287 , Rosanna, Victoria 3084",Volunteering,Endowment Australasia,3084,Rosanna,Banyule City Council,-37.742729,145.065277,"[Banyule City Council, Rosanna, Volunteering]",https://myclaimsolved.com/.../national-mutual-...
9,"1151 Glen Huntly Road, Glen Huntly, Victoria 3163",Volunteering,Glen Eira U3A Inc.,3163,Glen Huntly,Glen Eira City Council,-37.888870,145.041820,"[Glen Eira City Council, Glen Huntly, Voluntee...",https://gleneirau3a.org.au
10,"C/- Community Information Glen Eira, 256 Hawth...",Volunteering,Glen Eira Volunteer Information Service,3162,Caulfield,Glen Eira City Council,-37.880540,145.022230,"[Glen Eira City Council, Caulfield, Volunteering]",https://www.gleneira.vic.gov.au/.../community-...


In [39]:
volunteer_mel[volunteer_mel.duplicated(subset=['Name'])]

,Address,Business Category,Name,Postcode,Suburb,Municipality,Latitude,Longitude,Categories,Contact


## Write to file

In [40]:
volunteer_mel.to_csv("../Data/Volunteers.csv", index=False)